In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set() 
import warnings
warnings.filterwarnings(action='ignore')

plt.rcParams['font.family']= 'Malgun Gothic'  # 한글 처리
plt.rcParams['axes.unicode_minus'] = False  # -, $ 등등에 표현 특수기호로 버리지 말고 다 써줘라.

In [4]:
# plotly를 위한 임포트
import plotly as ply
import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.subplots import make_subplots # 서브플롯을 만들기 위헤
import plotly.graph_objects as go

import chart_studio
chart_studio.tools.set_credentials_file(username='jisu_asiae', api_key='TkLnzkkz6NPV1m3bnVm6')

ModuleNotFoundError: ignored

주요 Event:
* 코로나: 2019년도 12월부터 시작

# read data, preprocessing

In [ ]:
df1 = pd.read_csv('./datasets/블록별 성별연령대별 카드소비패턴.csv', encoding='cp949')
df1.info()

In [ ]:
df2 = pd.read_csv('./datasets/카드소비 업종코드.csv', encoding='cp949')
df2.info()

In [ ]:
df1['년'] = df1['기준년월(YM)'].astype('str').str[:4]
df1['월'] = df1['기준년월(YM)'].astype('str').str[4:]

In [ ]:
df1.rename(columns = {'서울시민업종코드(UPJONG_CD)' : '업종코드', '성별(GEDNER)' : '성별', '카드이용금액계(AMT_CORR)' : '금액', '카드이용건수계(USECT_CORR)' : '건수', '연령대별(AGE)' : '연령'}, inplace = True)
df2.rename(columns = {'업종코드(UPJONG_CD)' : '업종코드', '대분류(CLASS1)' : '대분류', '중분류(CLASS2)' : '중분류', '소분류(CLASS3)' : '소분류'}, inplace = True)
df2['업종코드'] = df2['업종코드'].str.upper()
mgdf = pd.merge(df1, df2, on='업종코드', how='left')

mgdf.info()

In [ ]:
df = mgdf[['대분류','중분류','소분류', '성별', '연령', '년', '월', '금액', '건수']]
df.head()

In [ ]:
df['대분류'] = df['대분류'].replace(['가전/가구','가정생활/서비스'],'가정생활')
df['대분류'] = df['대분류'].replace('교육/학원','교육')
df['대분류'] = df['대분류'].replace(['스포츠/문화/레저','여행/교통'],'문화')
df['대분류'] = df['대분류'].replace(['주유','자동차'],'교통')
df['대분류'] = df['대분류'].replace(['요식/유흥','음/식료품'],'식품')
df['대분류'] = df['대분류'].replace(['의류/잡화','미용'],'패션')
df['대분류'].value_counts()

## 이상치 처리

In [ ]:
df[(df['연령']=='10대')&(df['대분류']=='교통')]

In [ ]:
df[(df['성별']=='F')&(df['대분류']=='교통')&(df['연령']=='60대')&(df['년']=='2018')]

In [ ]:
df[(df['대분류']=='식품')&(df['연령']=='10대')&(df['성별']=='F')]

In [ ]:
df[(df['대분류']=='식품')&(df['연령']=='10대')]

In [ ]:
df[(df['대분류']=='식품')&(df['연령']=='10대')&(df['성별']=='F')]

In [ ]:
df[(df['년']=='2021')&(df['대분류']=='전자상거래')&(df['성별']=='M')&(df['연령']=='20대')]

In [ ]:
# drop
df = df.drop(df[(df['연령']=='10대')&(df['대분류']=='교통')].index.values[0], axis=0)

In [ ]:
idxv = df[(df['성별']=='F')&(df['대분류']=='교통')&(df['연령']=='60대')&(df['년']=='2018')].index.values[0]
df = df.drop(idxv, axis=0)

In [ ]:
idxv = df[(df['대분류']=='식품')&(df['연령']=='10대')&(df['성별']=='F')].index.values[0]
df = df.drop(idxv, axis=0)

In [ ]:
temp = df[(df['년']=='2021')&(df['대분류']=='전자상거래')&(df['성별']=='M')&(df['연령']=='20대')]
idxv =df[(df['년']=='2021')&(df['대분류']=='전자상거래')&(df['성별']=='M')&(df['연령']=='20대')].index.values[0]
df = df.drop(idxv, axis=0)
temp['금액'] = int(temp['금액'].values[0]/4)
df = pd.concat([df, temp], axis=0)
df[(df['년']=='2021')&(df['대분류']=='전자상거래')&(df['성별']=='M')&(df['연령']=='20대')]

In [ ]:
temp_df = df[(df['대분류']=='식품')&(df['연령']=='20대')&((df['소분류']=='한식')|(df['소분류']=='기타음/식료품'))].sort_values('년')
temp_df = temp_df[((temp_df['년']!='2018')|(temp_df['월']!='04'))]
temp_df = temp_df[((temp_df['년']!='2018')|(temp_df['월']!='07'))]
temp_df['연령']= '10대'
temp_df

In [ ]:
df = pd.concat([df, temp_df], axis=0)
df.info()

In [ ]:
df.info()

In [ ]:
df=df.sort_values(['년','월'])
df.head()

# 성별

In [ ]:
# 분석 결과, 2016-2023년도 까지 금액의 성별에 따른 금액의 총합과 건수의 평균으로 표시
# 건수의 경우, 여성이 높으나 금액은 남성이 높음.
meandata = df.groupby('성별')['금액'].mean()
ratio1 = meandata.values
labels1 = meandata.index.values

meandata2 = df.groupby('성별')['건수'].mean()
ratio2 = meandata2.values
labels2 = meandata2.index.values

plt.subplot(1,2,1)
plt.title('금액')
plt.pie(ratio1, labels=labels1, autopct='%.1f%%')

plt.subplot(1,2,2)
plt.title('건수')
plt.pie(ratio2, labels=labels2, autopct='%.1f%%')

plt.show()

In [ ]:
gdf = df.groupby('성별')[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

plt.subplot(2,1,1)
sns.barplot(data=gdf, x="성별", y="금액") 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x="성별", y="건수") 
plt.show()

In [ ]:
# 성별에 따른 금액과 건수를 대분류로 나눈 결과
# 남성의 경우 전자상거래에서 소비 금액이 아주 큼.
# 여성의 경우 교통에서 결제 건수가 많은 것으로 나타남. ??  --> 해당 이유 파악 필요.
gdf = df.groupby(['성별', '대분류'])[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

plt.figure(figsize=(15,10))
plt.subplot(2,1,1)
sns.barplot(data=gdf, x="성별", y="금액", hue='대분류') 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x="성별", y="건수", hue='대분류') 
plt.show()

# 연령별

In [ ]:
# 연령별, 금액, 건수
# 10대의 소비 금액이 높음.
# 건수의 경우 50개의 소비 건수가 가장 높으나, 금액은 높지 않음.
# 즉, 50개 소비 패턴이 상대적으로, 금액이 적은 소비를 자주 하는 것으로 보임.

gdf = df.groupby('연령')[['금액', '건수']].mean().sort_index()

plt.subplot(2,1,1)
sns.barplot(data=gdf, x=gdf.index, y="금액") 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x=gdf.index, y="건수") 
plt.show()

In [ ]:
gdf = df.groupby(['연령', '대분류'])[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

plt.figure(figsize=(15,10))
plt.subplot(2,1,1)
sns.barplot(data=gdf, x='연령', y="금액", hue='대분류') 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x='연령', y="건수", hue='대분류') 
plt.show()

# 10대의 교통과 식품 소비 금액이 매우 높음. 교통(자동차, 주유)이 높게 나오는 근거가 부족. 
# 20대의 전자상거래, 소비금액이 높고, 50대의 의료 비용이 높음.
# 60~70대의 경우, 교통 소비 금액은 높지 않으나, 건수는 높게 나옴. 교통에 적은 금액을 자주 사용하는 것으로 보임.

# 아래부터

In [ ]:
df.columns

# 연도별

---

In [ ]:
# 연도별 금액, 건수
# 2021년도, 건수는 높지 않으나, 금액이 높게 나옴.  (2021년도에 금액이 큰 소비가 발생함.)
# 2016년도에 금액과 건수 모두 높았다.
# 2021년도의 겨우 건수는 적으나, 금액이 높음. 2021년도에 사람들이 금액이 큰 소비를 한듯함.
# 매년 평균적으로 여성의 건수
gdf = df.groupby('년')[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

plt.subplot(2,1,1)
sns.barplot(data=gdf, x='년', y="금액") 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x='년', y="건수") 
plt.show()

In [ ]:
fig = make_subplots(rows=2, subplot_titles=['금액' , '건수'])

x_year = df['년'].unique()
y_data2 = df.groupby('년')['건수'].mean()
y_data = df.groupby('년')['금액'].mean()

fig.add_trace( go.Bar(x=x_year ,y=y_data),1, 1)
fig.add_trace( go.Bar(x=x_year, y=y_data2), 2,1)

fig.update_layout(width=600, height=700)

# py.plot(fig, filename = 'year_mc')
fig.show()

---

## 연도별 대분류별

In [ ]:
# 연도에 따른 연도별 금액, 건수 비교 차트 
# 2021년도에는 전자상거래를 큰금액 했지만 건수는 적음
# 2017년도에는 문화에 건수가 높지만 금액은 높지 않음
gdf = df.groupby(['년', '대분류'])[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

fig,axes = plt.subplots(2,1,figsize=(17,10))
sns.barplot(data=gdf, x='년', y="금액", hue='대분류',ax=axes[0]) 
axes[0].set_title('년도별 분류별 금액')

plt.subplot(2,1,2)
sns.barplot(data=gdf, x='년', y="건수", hue='대분류',ax=axes[1]) 
axes[1].set_title('년도별 분류별 건수')

plt.tight_layout()
plt.show()

### 건수

In [ ]:
list_data=[]
for ca in df['대분류'].unique():
    dfdf = df[df['대분류']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['건수'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=1400, height=400)

# py.plot(fig, filename = 'year_cat_count') #-> 차트 plotly에 저장
fig.show()

### 금액

In [ ]:
list_data=[]
for ca in df['대분류'].unique():
    dfdf = df[df['대분류']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['금액'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=1400, height=400)

# py.plot(fig, filename = 'year_cat_money') #-> 차트 plotly에 저장
fig.show()

---

## 연도별 성별 

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(8,8))
sns.barplot(data=df.sort_values('년'), x='년', y='건수', estimator='mean',errorbar=None,ax=axes[0],hue='성별')
axes[0].set_title('연도별 성별 건수')

sns.barplot(data=df.sort_values('년'), x= '년', y='금액', estimator='mean', errorbar=None,ax=axes[1],hue='성별')
axes[1].set_title('연도별 성별 금액')

plt.tight_layout()
plt.show()

### 건수

In [ ]:
list_data=[]
for ca in df['성별'].unique():
    dfdf = df[df['성별']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['건수'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=800, height=500)

# py.plot(fig, filename = 'year_sex_count') #-> 차트 plotly에 저장
fig.show()

### 금액

In [ ]:
list_data=[]
for ca in df['성별'].unique():
    dfdf = df[df['성별']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['금액'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=800, height=500)

# py.plot(fig, filename = 'year_sex_money') #-> 차트 plotly에 저장
fig.show()

---

## 연도별 연령별 

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(15,8))
sns.barplot(data=df.sort_values(['년','연령']), x='년', y='건수', estimator='mean',errorbar=None,ax=axes[0],hue='연령')
axes[0].set_title('연도별 성별 건수')

sns.barplot(data=df.sort_values(['년','연령']), x= '년', y='금액', estimator='mean', errorbar=None,ax=axes[1],hue='연령')
axes[1].set_title('연도별 성별 금액')

plt.tight_layout()
plt.show()

In [ ]:
ddff=df['연령'].sort_values() # 정렬을 위해
ddff.unique()

### 건수

In [ ]:
list_data=[]
for ca in ddff.unique():
    dfdf = df[df['연령']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['건수'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=1200, height=500)

# py.plot(fig, filename = 'year_age_count') #-> 차트 plotly에 저장
fig.show()

### 금액

In [ ]:
list_data=[]
for ca in ddff.unique():
    dfdf = df[df['연령']==ca].groupby('년').mean()
    data = go.Bar(x=df['년'].unique() ,y=dfdf['금액'], name=ca)
    list_data.append(data)
    
fig = go.Figure(data=list_data) # 생성된 Bar 클래스를 리스트로 만들어 data 인자로 설정
fig.update_layout(width=1200, height=500)

# py.plot(fig, filename = 'year_age_money') #-> 차트 plotly에 저장
fig.show()

---

---

# 월별

In [ ]:
gdf = df.groupby('월')[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

plt.subplot(2,1,1)
sns.barplot(data=gdf, x='월', y="금액") 
plt.subplot(2,1,2)
sns.barplot(data=gdf, x='월', y="건수") 
plt.show()

In [ ]:
fig = make_subplots(rows=2, subplot_titles=['금액' , '건수'])

x_month = df['월'].unique()
y_data2 = df.groupby('월')['건수'].mean()
y_data = df.groupby('월')['금액'].mean()

fig.add_trace( go.Bar(x=x_month ,y=y_data),1, 1)
fig.add_trace( go.Bar(x=x_month, y=y_data2), 2,1)

fig.update_layout(width=650, height=700)

# py.plot(fig, filename = 'month_mc')
fig.show()

----

## 월별 대분류별

In [ ]:
gdf = df.groupby(['월', '대분류'])[['금액', '건수']].mean().sort_index()
gdf = gdf.reset_index()

fig,axes = plt.subplots(2,1,figsize=(17,10))
sns.barplot(data=gdf, x='월', y="금액", hue='대분류',ax=axes[0]) 
axes[0].set_title('월별 분류별 금액')

plt.subplot(2,1,2)
sns.barplot(data=gdf, x='월', y="건수", hue='대분류',ax=axes[1]) 
axes[1].set_title('월별 분류별 건수')

plt.tight_layout()
plt.show()

---

## 월별 성별 

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(9,10))
sns.barplot(data=df.sort_values('월'), x='월', y='건수', estimator='mean',errorbar=None,ax=axes[0],hue='성별')
axes[0].set_title('월별 성별 건수')

sns.barplot(data=df.sort_values('월'), x= '월', y='금액', estimator='mean', errorbar=None,ax=axes[1],hue='성별')
axes[1].set_title('월별 성별 금액')

plt.tight_layout()
plt.show()

---

## 월별 연령별 

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(20,10))
sns.barplot(data=df.sort_values(['월','연령']), x='월', y='건수', estimator='mean',errorbar=None,ax=axes[0],hue='연령')
axes[0].set_title('월별 성별 건수')

sns.barplot(data=df.sort_values(['월','연령']), x= '월', y='금액', estimator='mean', errorbar=None,ax=axes[1],hue='연령')
axes[1].set_title('월별 성별 금액')

plt.tight_layout()
plt.show()